# Error Handling

Rust 不同于别的语言，使用**枚举**作为错误处理的方式

- 枚举`enum`是一种类型，可以包含多个变量之一

In [ ]:
enum TrafficLightColor {    // Type
    Red,                    // variants
    Yellow,
    Green,
}

let current_state = TrafficLightColor::Green;

- Rust: `match`表达式类似于 C/C++/Java 中的 `switch` 语句，但是必须覆盖所有可能的值

如果没有全部覆盖，则会报错，如下所示

In [3]:
enum TrafficLightColor {    // Type
    Red,                    // variants
    Yellow,
    Green,
}


fn drive(light_state: TrafficLightColor) {
    match light_state {
        TrafficLightColor::Green => println!("zoom zoom!"),
        TrafficLightColor::Red => println!("sitting like a boulder!"),
    }
}

Error: non-exhaustive patterns: `TrafficLightColor::Yellow` not covered

- 如果你只对其中几个感兴趣，可以使用默认绑定来捕获所有其他情况。

```rust
match light_state {
    TrafficLightColor::Green => println!("zoom zoom!"),
    _ => println!("do not pass go"), // default binding. 
    // DO THIS in all other cases
}

- 与大多数语言的枚举不同，Rust 枚举可以存储任意数据

In [ ]:
enum Location {
    Coordinates(f32, f32),
    Address(String),
    Unknown,
}

比如：
想要存储某物的位置，并希望有选项表示它：
- 经纬度坐标（32位浮点数）
- 地址（值为字符串）
- 未知（没有任何关联数据）

可以使用 `match` 表达式从变量中提取数据

In [5]:
enum Location {
    Coordinates(f32, f32),
    Address(String),
    Unknown,
}

fn print_location(loc: Location) {
    match loc {
        Location::Coordinates(lat, long) => {
            println!("Person is at ({}, {})", lat, long);
        },
        Location::Address(addr) => {
            println!("Person is at {}", addr);
        },
        Location::Unknown => println!("Location unknown!"),
    }
}

print_location(Location::Address("yeugdauabi233!".to_string()));

Person is at yeugdauabi233!


如果使用枚举来清晰地表示成功返回/错误会怎样呢？

- 如果函数成功运行，则返回OK (任何返回值)
- 如果发生错误，则返回Err (某个错误对象)

In [ ]:
// Rust 标准库中的一部分
enum Result<T, E> {
    OK(T),
    Err(E),
}

// T 是成功时要存储在OK中的值的类型
// E 是错误时要存储在Err中的值的类型

In [ ]:
// u32 成功返回的类型，str 错误返回的类型
fn gen_num_sometimes() -> Result<u32, &'stack str> {
    if get_random_num() > 10 {
        OK(get_random_num())
    } else {
        Err("number is less than 11")
    }
}

fn main() {
    match gen_num_sometimes() {
        Ok(num) => println!("get number: {}", num),
        Err(message) => println!("Operation failied: {}", message),
    }
}

## 对比 C 语言的错误

在 C 语言错误处理方面遇到的两个主要问题：
- 很容易错过错误
- 正确的错误处理太冗长了（需要额外的代码来传播错误）

Rust 解决了第一个问题：从函数签名中可以明显看出哪些函数可能返回错误，而且（由于枚举规则）编译器将验证是否对返回的错误进行了处理。

但是第二个问题还是存在，错误处理仍然太冗长了

In [ ]:
fn read_file(filename: &str) -> Result<String, io::Error> {
    let mut s = String::new();

    let result = File::open(filename);

    let mut f = match result {
        Ok(file) => file,
        Err(e) => return Err(e),
    }

    match f.read_to_string(&mut s) {
        Ok(_) => Ok(s),
        Err(e) => Err(e),
    }
}

### ? 操作

假设我们有一个函数 `helper_function() -> Result<T, E>`

`let val: T = helper_function()?`
意味着：
- 如果函数返回 Ok(some value), 那么就将 val 设置成那个值
- 如果函数返回 Err(some error)，停止并且返回/传播那么错误

因此上面代码可以改写如下：

In [ ]:
fn read_file(filename: &str) -> Result<String, io::Error> {
    let mut s = String::new();

    let mut f = File::open(filename)?;
    // 如果文件成功打开了，那么存储文件到 f
    // 如果一个错误发生了，那么就停止并且返回错误给调用者

    f.read_to_string(&mut s)?;
    Ok(s)
}

甚至可以更短

In [ ]:
fn read_file(filename: &str) -> Result<String, io::Error> {
    let mut s = String::new();
    File::open(filename)?.read_to_string(&mut s);
    Ok(s)
}

## Panics

- 如果遇到不希望传播或者处理的错误呢？
    - 可能是一个严重的，不可恢复的错误
    - 可能是无法预料的错误，也不想花费精力去处理
    - 比如：如果是一个终端程序，无法从终端读取输入，那么实际上没有什么优雅的解决方法

- `panic!`宏会立即崩溃程序并显示错误消息
    ```rust
    if sad_times() {
        panic!("Sad times!");
    }
    ```

- `Result::unwrap()` 和 `Result::expect()` 允许我们去提取从Ok
()返回的值，如果遇到 Err 则 panic

### `unwrap()` 和 `expect()`

In [ ]:
// File::open 返回 Result: Ok(file) or Err(error)
// unwrap 意味着：
// - 如果结果是 Ok：存储枚举的值到 file
// - 如果结果是 Err（打开文件失败）：panic(终止程序)

// 如果打开文件失败，则 Panic
let mut file = File::open(filename).unwrap();

// expect 和 unwrap 一样，但是当发生 panic 时，允许打印更多的错误描述信息
let mut file = File::open(filename).expect("Failed to open file");

// 

## 处理 nulls

为什么 Null 值如此危险？
- 它们给程序员带来了巨大的负担，每当有一个指针时，都需要思考，它是否为空（NULL）？
- 静态分析其无法在不被错误警告淹没的情况下警告所有的 NULL

我们应该怎么做呢？

```rust
enum Option<T> {
    None,
    Some(T),
}
```

In [ ]:
fn feeling_lucky() -> Option<String> {
    if get_random_num() > 10 {
        Some(String::from("I'm feeling lucky!"))
    } else {
        None
    }
}

match feeling_lucky() {
    Some(message) => {
        println!("Got message: {}", message);
    },
    None => {
        println!("No message returned :-/");
    }
}

In [ ]:
fn feeling_lucky() -> Option<String> {
    if get_random_num() > 10 {
        Some(String::from("I'm feeling lucky!"))
    } else {
        None
    }
}

// check if is_none/is_some():
if feeling_lucky().is_none() {
    println!("Not feeling lucky :(")
}

// unwrap/expect 在这里也生效
let message = feeling_lucky().unwrap();

let message = feeling_lucky().expect("feeling_lucky failed us!");

// 如果返回 None 时，可以提供一个默认的值
let message = feeling_lucky().unwrap_or("Not feeling lucky :(".to_string());

// ? 同样在函数返回时生效
let expanded_message: String = feeling_lucky()? + " Are you?";